In [13]:
'''
This code converts the format of the HRRR data files. Originally, the HRRR files are in GRIB2 format. 
This code converts into netCDF from GRIB2 format. 
'''

'\n04/01/2025: This file is created\n'

In [ ]:
import os
import pygrib
import xarray as xr

# Define directories
hrrr_directory = r"D:\2Courses\8902\data\hrrr\20231026\hrrr"  # Source directory containing HRRR GRIB2 files
output_directory = r"D:\2Courses\8902\reprojected_data\converted_HRRR"  # Base output directory for converted files

# Ensure the output base directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

def create_daily_folder_and_convert(grib_file, day_folder):
    """
    Convert GRIB2 file to netCDF and save in the respective day's folder.
    """
    try:
        # Use pygrib to read GRIB2 file
        grbs = pygrib.open(grib_file)
        data_vars = {}
        coords = {}

        # Extract relevant data and coordinates
        for grb in grbs:
            variable_name = grb.parameterName
            lat, lon = grb.latlons()
            data = grb.values

            # Store data in data_vars
            data_vars[variable_name] = (["lat", "lon"], data)

            # Set coordinates
            coords["lat"] = lat[:, 0]  # Latitude
            coords["lon"] = lon[0, :]  # Longitude

        # Create xarray Dataset
        dataset = xr.Dataset(data_vars, coords)
        dataset.attrs["description"] = "Converted from GRIB2 to netCDF"

        # Generate netCDF file name
        netcdf_file = os.path.join(day_folder, f"{os.path.splitext(os.path.basename(grib_file))[0]}.nc")
        
        # Save dataset to netCDF
        dataset.to_netcdf(netcdf_file)
        print(f"Converted: {grib_file} -> {netcdf_file}")
    except Exception as e:
        print(f"Error converting {grib_file}: {e}")

def process_hrrr_files(input_dir, output_dir):
    """
    Organize files by day and convert GRIB2 files to netCDF.
    """
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".grib2"):
                grib_file = os.path.join(root, file)
                
                # Extract day folder from the path and create a corresponding folder in output directory
                day_folder_name = os.path.basename(os.path.dirname(grib_file))
                day_folder_path = os.path.join(output_dir, day_folder_name)

                if not os.path.exists(day_folder_path):
                    os.makedirs(day_folder_path)

                # Convert GRIB2 to netCDF and save in the day's folder
                create_daily_folder_and_convert(grib_file, day_folder_path)
    
    # Delete .idx files from the input directory
    delete_idx_files(input_dir)

def delete_idx_files(folder_path):
    """
    Deletes all .idx files in the specified folder and its subdirectories.
    """
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".idx"):
                idx_file = os.path.join(root, file)
                os.remove(idx_file)
                print(f"Deleted: {idx_file}")

# Run the script for the HRRR directory
print("Processing HRRR files...")
process_hrrr_files(hrrr_directory, output_directory)
print(f"All converted netCDF files saved to respective daily folders in: {output_directory}")


Processing HRRR files...
